## Finetuning YOLOv11 with Buffalo and Camel labels.
For installation and setup, view the [README](./README.md) before running this.

As mentioned in the README, we will be using Roboflow for labelling our dataset.

The Roboflow project is [available here](https://app.roboflow.com/stuff-avvl2/yolo-fine-y444l).

## Download Annotated Dataset
We will download the versioned dataset directly from Roboflow to the target directory.

v8 format is used as it is widely supported.

In [12]:
import os
from roboflow import Roboflow
from tqdm import tqdm
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()
target_path = Path("yolo-fine-1/")

if target_path.exists() and len(os.listdir(target_path)) > 5:
    print(f"Dataset already found at: {target_path}. Skipping download.")
else:
    API_KEY = os.getenv("ROBOFLOW_API_KEY")
    rf = Roboflow(api_key=API_KEY)
    project = rf.workspace("stuff-avvl2").project("yolo-fine-y444l")
    dataset = project.version("1").download("yolov8")

Dataset already found at: yolo-fine-1. Skipping download.


## Training the Model

### Setup the logging

Ultralytics support logging to wandb, comet.ml and tensorboard, out of the box. Here we only enable wandb.

You need to create an account at [wandb](https://wandb.ai) and get the API key from https://wandb.ai/authorize.

Run `wandb login` in the terminal after activating the conda environment.

In [1]:
from ultralytics import settings

settings.update({"wandb": True,
                 "tensorboard": False})

### Training

We specify the path to data.yaml file, and train with a batch size of 15, and we also save the checkpoint at each epoch (save_period=1). We assume here you are connected to a GPU, hence we can specify the device to use as `device=0` to select the first GPU.  We specify the project name as balloon, this will create a folder called `balloon` to store the weights and various training artifacts such as F1, PR curves, confusion matrics, training results (loss, mAP, etc).

For a complete listing of train settings, you can see [here](https://docs.ultralytics.com/modes/train/#train-settings).

You can also specify the type of data [augmentation](https://docs.ultralytics.com/modes/train/#augmentation-settings-and-hyperparameters)  you want as part of the train pipeline.

You can monitor your training progress at wandb (the link is given in the train output below)

My GPU specs are not the best (no CUDA support), hence we will defer training logic to the terminal via a [Python script](train_model.py) to prevent out of memory issues.
> Even if you have good hardware, please do not run with workers set to a low value or the Jupyter kernel might crash.

Refer to the Python script for the applied training logic. The script content is appended here for convenience.

Training output to `stdout` is logged within this [log file](train_model.log).

In [ ]:
# import multiprocessing
# import sys
# from ultralytics import YOLO
# from ultralytics import settings
# import os

# LOG_FILE = "train_model.log"
# MAX_WORKERS = max(1, multiprocessing.cpu_count() - 2)
# BATCH_SETTING = 8
# LR0 = 0.001 
# LRF = 0.0001
# EPOCHS = 35
# PATIENCE = 5
# DET = False

# # Custom Class to output to stdout + log file
# class Tee(object):
#     """Duplicates stdout output to both a terminal stream and a file."""
#     def __init__(self, filename, mode="a", stream=sys.stdout):
#         self.terminal = stream
#         self.log_file = open(filename, mode)

#     def write(self, message):
#         self.terminal.write(message)
#         self.log_file.write(message)

#     def flush(self):
#         self.terminal.flush()
#         self.log_file.flush()

#     def __enter__(self):
#         self.original_stdout = sys.stdout
#         sys.stdout = self
#         return self

#     def __exit__(self, exc_type, exc_val, exc_tb):
#         sys.stdout = self.original_stdout
#         self.log_file.close()

# settings.update({
#     "wandb": True,
#     "tensorboard": False,
# })

# print(f"Starting YOLOv11 training. Output will be saved to {LOG_FILE} and console.")

# with Tee(LOG_FILE, mode='a') as t:
#     print("-" * 50)
#     print(f"Training session started at: {os.uname().nodename}")
#     print(f"Using {MAX_WORKERS} multiprocessing workers (workers argument).")
#     print(f"Using batch={BATCH_SETTING}.")
#     print("-" * 50)
    
#     try:
#         model = YOLO("yolo11s.pt")

#         result = model.train(
#             data="yolo-fine-1/data.yaml",
#             epochs=EPOCHS,
#             patience=PATIENCE,
#             save_period=1,
#             batch=BATCH_SETTING,
#             workers=MAX_WORKERS,
#             device="0",
#             project='yolo-fine',
#             plots=True,
#             optimizer="AdamW",
#             lr0=LR0,
#             lrf=LRF,
#             deterministic=DET
#         )

#     except Exception as e:
#         print(f"\n[ERROR] An exception occurred during training: {e}", file=sys.stderr)

# print(f"Training finished. Full log available in {LOG_FILE}.")

### Benchmarks
View [training benchmarks here](./benchmarks.md).

## Wandb Metrics
Best found metrics on WanDB are documented here.

*metrics*

![Wandb Metrics](./results/v6-metrics.png)

*training loss*

![Wandb Metrics](./results/v6-train-loss.png)

*validation loss*

![Wandb Metrics](./results/v6-val-loss.png)

*average precision*
![Wandb Metrics](./results/BoxPR_curve.png)

After training, we can run the best model (using the best checkpoint) against the validation dataset to see the overall model performance on validation set.

You should see around 0.83 for mAP50, and 0.65 for mAP50-95.

Alternatively, you can use the [metrics script](./get_local_metrics.py).

In [1]:
from ultralytics import YOLO

# modify this number to correspond to train, train2 and so on directories
train_ver = 2

model = YOLO(f"yolo-fine/train{train_ver}/weights/best.pt")

validation_results = model.val(data="yolo-fine-1/data.yaml", device="0")

print("\n--- Key Metrics ---")
print(f"Overall mAP50 (map50): {validation_results.box.map50:.4f}")
print(f"Overall mAP50-95 (map): {validation_results.box.map:.4f}")
print(f"Mean Precision (mp): {validation_results.box.mp:.4f}")
print(f"Mean Recall (mr): {validation_results.box.mr:.4f}")

Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2050, 3771MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 235.7±124.6 MB/s, size: 47.0 KB)
val: Scanning /home/data/Desktop/projects/yolo-finetune/yolo-fine-1/valid/labels.cache... 164 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 164/164 422.3Kit/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 182, len(boxes) = 308. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 4.2it/s 2.6s0.3s
                   all        164        308       0.85      0.799      0.839      0.656
               buffalo         76        158      0.868      0.778      0.

## Export and Deployment
Our model is in pytorch format (.pt). You can export the model to various format, e.g. TorchScript, ONNX, OpenVINO, TensorRT, etc. depending on your use case, and deployment platform (e.g. CPU or GPU, etc)

You can see the list of [supported formats](https://docs.ultralytics.com/modes/export/#export-formats)  and the option they support in terms of further optimization (such as imagesize, int8, half-precision, etc) in the ultralytics site.

Ultralytics provide a utility function to benchmark your model using different supported formats automatically. You can uncomment the code in the following code cell to see the benchmark result. If you are benchmarking for CPU only, the change the `device=0` to `device='cpu'`.  

**This will fail as my GPU does not support CUDA.**

In [ ]:
# from ultralytics.utils.benchmarks import benchmark

# # Benchmark on GPU (device=0 means the 1st GPU device)
# benchmark(model="yolo-fine/train2/weights/best.pt", data="yolo-fine-1/data.yaml", imgsz=640, half=False, device=0)

In the following code, we export it as OpenVINO. OpenVINO is optimized for inference on Intel CPUs and since we will use the model later on to do inference on local Windows machine (which runs Intel chip), we will export it as OpenVINO format. We also specify using int8 quantization, which results in faster inference, at the cost of accuracy.

For more information on OpenVINO, go to the [official documentation](https://docs.openvino.ai/2024/index.html).

After export, you can find the openvino model in `yolo-fine\train2\weights\best_openvino_model` directory.

In [3]:
from ultralytics import YOLO
model = YOLO("yolo-fine/train2/weights/best.pt")
exported_path = model.export(format="openvino", int8=True)

Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1+cu128 CPU (12th Gen Intel Core i7-1255U)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from 'yolo-fine/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (18.3 MB)

OpenVINO: starting export with openvino 2025.4.0-20398-7a975177ff4-releases/2025/4...
OpenVINO: collecting INT8 calibration images from 'data=coco8.yaml'
Fast image access ✅ (ping: 0.0±0.0 ms, read: 336.1±113.2 MB/s, size: 54.0 KB)
Scanning /home/data/Desktop/projects/yolo-finetune/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 4/4 8.3Kit/s 0.0s
WARNING ⚠️ OpenVINO: >300 images recommended for INT8 calibration, found 4 images.
INFO:nncf:15 ignored nodes were found by patterns in the NNCFGraph
INFO:nncf:1 ignored nodes were found by t

OpenVINO: export success ✅ 8.7s, saved as 'yolo-fine/train2/weights/best_int8_openvino_model/' (9.8 MB)

Export complete (9.2s)
Results saved to /home/data/Desktop/projects/yolo-finetune/yolo-fine/train2/weights
Predict:         yolo predict task=detect model=yolo-fine/train2/weights/best_int8_openvino_model imgsz=640 int8 
Validate:        yolo val task=detect model=yolo-fine/train2/weights/best_int8_openvino_model imgsz=640 data=yolo-fine-1/data.yaml int8 
Visualize:       https://netron.app


In [4]:
import ultralytics

from ultralytics import YOLO

from PIL import Image

source = './images/camel_stock_image.jpg'
model = YOLO("./yolo-fine/train2/weights/best_int8_openvino_model", task='detect')
result = model(source, conf=0.5, iou=0.6)

for i, r in enumerate(result):
    print(r)
    im_bgr = r.plot()
    im_rgb = Image.fromarray(im_bgr[..., ::-1])
    r.show()
    r.save(filename=f"metric_results{i}.jpg")

Loading ./yolo-fine/train2/weights/best_int8_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference on CPU...

image 1/1 /home/data/Desktop/projects/yolo-finetune/images/camel_stock_image.jpg: 640x640 2 camels, 50.4ms
Speed: 2.1ms preprocess, 50.4ms inference, 8.2ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'buffalo', 1: 'camel'}
obb: None
orig_img: array([[[204, 181, 159],
        [204, 181, 159],
        [204, 181, 159],
        ...,
        [213, 190, 168],
        [213, 190, 168],
        [213, 190, 168]],

       [[204, 181, 159],
        [204, 181, 159],
        [204, 181, 159],
        ...,
        [213, 190, 168],
        [213, 190, 168],
        [213, 190, 168]],

       [[204, 181, 159],
        [204, 181, 159],
        [205, 182, 160],
        ...,
        [214, 191, 169],
        [213, 190

## Streaming

We can also do real-time detection on a video or camera stream.
### Detect and write to a video file 

In [5]:
from ultralytics import YOLO
import cv2
from tqdm import tqdm
from pathlib import Path
import os

def write_video(video_in_filepath, video_out_filepath, model):
    # Open the video file

    video_reader = cv2.VideoCapture(video_in_filepath)

    nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = video_reader.get(cv2.CAP_PROP_FPS)

    video_writer = cv2.VideoWriter(video_out_filepath,
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            fps,
                            (frame_w, frame_h))

    # Loop through the video frames
    for i in tqdm(range(nb_frames)):
        # Read a frame from the video
        success, frame = video_reader.read()

        if success:
            # Run YOLO inference on the frame on GPU Device 0
            results = model(frame, device=0)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Write the annotated frame
            video_writer.write(annotated_frame)

    video_reader.release()
    video_writer.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

video_in_file = "./images/buffalo_stock_footage.mp4"
video_out_file = os.path.join('./results','buffalo_stock_footage_detected' + '.mp4')
model = YOLO("./yolo-fine/train2/weights/best_int8_openvino_model", task='detect')
write_video(video_in_file, video_out_file, model)

  0%|                                                                                                                            | 0/249 [00:00<?, ?it/s]

Loading ./yolo-fine/train2/weights/best_int8_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference on CPU...

0: 640x640 1 buffalo, 45.8ms
Speed: 3.0ms preprocess, 45.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


  0%|▍                                                                                                                   | 1/249 [00:00<01:32,  2.68it/s]


0: 640x640 1 buffalo, 44.7ms
Speed: 2.8ms preprocess, 44.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 buffalo, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


  1%|█▍                                                                                                                  | 3/249 [00:00<00:35,  7.00it/s]

...

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 245/249 [00:17<00:00, 11.24it/s]


0: 640x640 1 buffalo, 73.9ms
Speed: 2.8ms preprocess, 73.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 buffalo, 75.5ms
Speed: 2.3ms preprocess, 75.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 247/249 [00:17<00:00, 11.07it/s]


0: 640x640 1 buffalo, 72.1ms
Speed: 2.2ms preprocess, 72.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 buffalo, 72.8ms
Speed: 2.1ms preprocess, 72.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 249/249 [00:18<00:00, 13.80it/s]


## Results of Inference and Streaming
You can find the metrics [here](https://github.com/bladeacer/yolo-finetune/tree/main/results).